In [37]:
##############   Vanilla Start  ~#################
# To import libraries and function to show images

import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!


def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()
    
    

In [39]:
# Will use b&w images (1channel) 10 as input noise size and 64 as the base of the hidden dimensions
#Philosophy a bit different from thje previous week, it is all much more encapsulated here, better. 
#We will just work with an instance of the class generator with the generator defined inside. 
class Generator (nn.Module):
    def __init__(self, z_dim = 10, im_chan =1, hidden_dim = 64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.gen = nn.Sequential(self.make_gen_block(z_dim, hidden_dim*4),
                self.make_gen_block(hidden_dim*4,hidden_dim*2,kernel_size=4,stride=1),
                self.make_gen_block(hidden_dim*2,hidden_dim),
                self.make_gen_block(hidden_dim,im_chan,kernel_size=4,final_layer=True))
        
    def make_gen_block (self, input_channels, output_channels, kernel_size = 3, stride = 2, final_layer = False):
        
        if not final_layer:
            return nn.Sequential(nn.ConvTranspose2d(input_channels,output_channels,kernel_size,stride),
                                nn.BatchNorm2d(output_channels),
                                nn.ReLU(inplace=True))
        else:
            return nn.Sequential(nn.ConvTranspose2d(input_channels,output_channels,kernel_size,stride),
                                nn.Tanh())
        
    def unsqueeze_noise(self,noise):
        #transform noise to get a copy with heigh and width =1. The channels are the z_dim to allow it to match winth the first convolution layer
        return noise.view(len(noise),self.z_dim,1,1)
    
    def forward(self, noise):
        x = self.unsqueeze_noise(noise)
        return self.gen(x)
    def get_noise(n_samples, z_dim, device = 'cpu'):
        return torch.randn(n_samples, z_dim, device = device)
        
        



In [46]:
class Discriminator(nn.Module):
    def __init__(self, im_chan=1,hidden_dim=16):
        super(Discriminator,self).__init__()
        self.disc = nn.Sequential(self.make_disc_block(im_chan,hidden_dim),
                                 self.make_disc_block(hidden_dim,hidden_dim*2),
                                 self.make_disc_block(hidden_dim*2,1,final_layer=True))
        
    def make_disc_block(self, input_channels, output_channels, kernel_size=4, stride = 2, final_layer=False):
        if not final_layer:
            return nn.Sequential(nn.Conv2d(input_channels, output_channels, kernel_size, stride),
                                nn.BatchNorm2d(output_channels),
                                nn.LeakyReLU(0.2, inplace=True))
        else:
            return nn.Sequential(nn.Conv2d(input_channels,output_channels, kernel_size, stride))
            
    def forward(self, image):
        disc_pred = self.disc(image)
        return disc_pred.view(len(disc_pred),-1)

In [49]:
criterion = nn.BCEWithLogitsLoss() #it includes final lauyer activaction function 
display_step = 500
z_dim = 64
batch_size = 128
lr = 0.0002
#for adam optimizer
beta_1 = 0.5
beta_2 = 0.999

device = 'cpu'

gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(),lr=lr,betas=(beta_1,beta_2))

disc = Discriminator().to(device)
disc_opt = torch.optim.Adam(gen.parameters(),lr=lr,betas=(beta_1,beta_2))



In [50]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,),(0.5,))])

dataloader  = DataLoader(MNIST('.',download=True,transform = transform),
                        batch_size=batch_size,
                        shuffle =True)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [52]:
#Initialize wights
def weights_init(m):
    if isinstance(m,nn.Conv2d) or isinstance(m,nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight,0.0,0.02)
    if isinstance(m,nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight,0.0,0.02)
        torch.nn.init.normal_(m.bias,0)

gen.apply(weights_init)
disc.apply(weights_init)

Discriminator(
  (disc): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(32, 1, kernel_size=(4, 4), stride=(2, 2))
    )
  )
)

In [56]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    fake_noise = get_noise(num_images, z_dim, device = device)
    fake = gen(fake_noise)#not needed .forward()
    
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred,torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred,torch.zeros_like(disc_real_pred))
    disc_loss = (disc_fake_loss+disc_real_loss)/2
    return disc_loss


def get_gen_loss (gen, disc, criterion, num_images, z_dim, device):
    fake_noise = get_noise(num_images, z_dim, device = device)
    fake = gen(fake_noise)
    
    disc_fake_pred = disc(fake)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss

In [58]:
n_epochs = 50
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0

for epoch in range(n_epochs):
    for real,_ in tqdm(dataloader):
        cur_batch_size = len(real)
        real = real.to(device)
        
        disc_opt.zero_grad()
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size,z_dim, device)
        disc_loss.backward(retain_graph = True)
        disc_opt.step()
        
        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step() 
        
        mean_discriminator_loss += disc_loss.item() / display_step
        mean_generator_loss += gen_loss.item() / display_step
        
        ## Visualization code ##
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            show_tensor_images(fake)
            show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1        

NameError: name 'dataloader' is not defined